<a href="https://colab.research.google.com/github/bhavinpatil/Question-Answering-System/blob/main/questionanswering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import torch
import numpy as np

In [4]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [5]:
tokenizer_for_bert = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

### **Function for question answering**

In [6]:
def bert_question_answer(question, passage, max_len=500):

  input_ids = tokenizer_for_bert.encode(question, passage, max_length = max_len, truncation=True)

  sep_index = input_ids.index(102)
  len_question = sep_index + 1
  len_passage = len(input_ids)-len_question

  segment_ids = [0]*len_question + [1]*(len_passage)

  tokens = tokenizer_for_bert.convert_ids_to_tokens(input_ids)

  start_token_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))[0]
  end_token_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))[1]

  start_token_scores = start_token_scores.detach().numpy().flatten()
  end_token_scores = end_token_scores.detach().numpy().flatten()

  answer_start_index = np.argmax(start_token_scores)
  answer_end_index = np.argmax(end_token_scores)

  start_token_scores = np.round(start_token_scores[answer_start_index], 2)
  end_token_scores = np.round(end_token_scores[answer_end_index], 2)

  answer = tokens[answer_start_index]
  for i in range(answer_start_index + 1, answer_end_index + 1):
    if tokens[i][0:2] == '##':
      answer+=tokens[i][2:]
    else:
      answer+=' '+tokens[i]
  

  if(answer_start_index == 0) or (start_token_scores < 0) or (answer == '[SEP') or (answer_end_index < answer_start_index):
    answer = "Sorry!, I could now find the answer in the passage."
  
  return (answer_start_index, answer_end_index, start_token_scores, end_token_scores, answer)


bert_question_answer("What harry discovers", "Harry Potter and the Philosopher's Stone, Harry lives in a cupboard under the stairs in the house of the Dursleys, his aunt, uncle and cousin, Dudley. The Dursleys consider themselves perfectly normal, but at the age of 11, Harry discovers that he is a wizard. He meets a half-giant named Hagrid who invites him to attend the Hogwarts School of Witchcraft and Wizardry. Harry learns that as a baby, his parents were murdered by the dark wizard Lord Voldemort. When Voldemort attempted to kill Harry, his curse rebounded and Harry survived with a lightning-shaped scar on his forehead.")

(58, 64, 4.61, 4.62, 'harry discovers that he is a wizard')

In [11]:
passage = """Harry Potter and the Chamber of Secrets describes Harry's second year at Hogwarts. Students are attacked and petrified by an unknown creature; wizards of Muggle parentage are the primary targets. The attacks appear related to the Chamber of Secrets, a fifty-year-old mystery at the school. Harry discovers an ability to speak the snake language Parseltongue, which he learns is rare and associated with the Dark Arts. When Hermione is attacked, Harry and Ron uncover the chamber's secrets and enter it. Harry discovers that the chamber was opened by Ron's younger sister, Ginny Weasley, who was possessed by an old diary in her belongings. The memory of Tom Marvolo Riddle, Voldemort's younger self, resided inside the diary and unleashed the basilisk, an ancient monster that kills those who make direct eye contact. Harry draws the Sword of Gryffindor from the Sorting Hat, slays the basilisk and destroys the diary.
In the third novel, Harry Potter and the Prisoner of Azkaban, Harry learns that he is targeted by Sirius Black, an escaped convict who allegedly assisted in his parents' murder. As Harry struggles with his reaction to the dementors – creatures guarding the school that feed on despair – he reaches out to Remus Lupin, a new professor who teaches him the Patronus charm. On a windy night, Ron is dragged by a black dog into the Shrieking Shack; Harry and Hermione follow. The dog is revealed to be Sirius Black. Lupin enters the shack and explains that Black was James Potter's best friend; he was framed by another friend of James', Peter Pettigrew, who hides as Ron's pet rat, Scabbers. As the full moon rises, Lupin transforms into a werewolf and bounds away; the group chase after him but are surrounded by dementors. They are saved by a mysterious figure who casts a stag Patronus. This is later revealed to be a future version of Harry, who traveled back in time with Hermione using the Time Turner. The duo help Sirius escape on a Hippogriff."""

print(f'Length of the passage: {len(passage.split())} words')

question1 = "The chamber was opened by whom"
print('\nQuestion 1: ', question1)
_, _, _, _, ans = bert_question_answer(question1, passage)
print('\nAnswer from BERT: ', ans, '\n')


question2 = "What ability harry discover in his second year at Hogwarts"
print('\nQuestion 1: ', question2)
_, _, _, _, ans = bert_question_answer(question2, passage)
print('\nAnswer from BERT: ', ans, '\n')


question3 = "Who was allegedly assisted in Harry's parents' murder"
print('\nQuestion 1: ', question3)
_, _, _, _, ans = bert_question_answer(question3, passage)
print('\nAnswer from BERT: ', ans, '\n')


question4 = "Harry and Hermione traveled back using what"
print('\nQuestion 1: ', question4)
_, _, _, _, ans = bert_question_answer(question4, passage)
print('\nAnswer from BERT: ', ans, '\n')



Length of the passage: 334 words

Question 1:  The chamber was opened by whom

Answer from BERT:  ginny weasley 


Question 1:  What ability harry discover in his second year at Hogwarts

Answer from BERT:  harry discovers an ability to speak the snake language parseltongue 


Question 1:  Who was allegedly assisted in Harry's parents' murder

Answer from BERT:  sirius black 


Question 1:  Harry and Hermione traveled back using what

Answer from BERT:  time turner 

